## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif
####  Output: *_label_ts.pkl

In [1]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess
from eeg_toolkit import source_localization as sl

0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage


### Paths

In [2]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

[-2.5,0.0,2.5]


In [3]:
# Read in processed raw data
data_dir = Path("../../Data")
# processed_data_path = data_dir / 'Processed Data/'
processed_data_path = data_dir / 'Resampled'

### Subject IDs

In [4]:
# 1 means chronic pain, 0 means control
sub_ids = {
    '5186': 1,
    '5295': 1,
    '5648': 0,
    '5675': 0,
    '5873': 0,
    '6100': 0,
    '6106': 0,
    '6310': 1,
}

In [5]:
print(f"Chronics: {len([k for k,v in sub_ids.items() if v == 1])}") 
print(f"Controls: {len([k for k,v in sub_ids.items() if v == 0])}")
print(f"Total: {len(sub_ids)}")

Chronics: 3
Controls: 5
Total: 8


In [6]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # CONTROLS
             # lateral occipital
             'lateraloccipital-lh', # Left Visual Cortex
             'superiortemporal-lh', # Left Auditory Cortex
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
             # CONTROLS
             'lateraloccipital-rh', # Right Visual Cortex
             'superiortemporal-rh', # Right Auditory Cortex
]

### Compute STCs

In [7]:
nan_subjects=[]

methods = [
    'MNE',
    # 'dSPM',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    for save_stc_mat in [
        False,
        # True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
            
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            # zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EO_resting_save_path,zscored_epochs_save_path] 
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            if os.path.exists(os.path.join(EO_resting_save_path, f"{sub_id}_eyes_open.pkl")):
                continue
        
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = True

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = sl.to_source(sub_id,
                                                            processed_data_path,
                                                            zscored_epochs_save_path,
                                                            EC_resting_save_path,
                                                            EO_resting_save_path,
                                                            roi_names,
                                                            times_tup,
                                                            method=method,
                                                            return_zepochs = False, # No epochs for Lupus
                                                            return_EC_resting = False,
                                                            return_EO_resting = True,
                                                            average_dipoles=average_dipoles,
                                                            save_stc_mat=save_stc_mat,
                                                            save_inv=save_inv,)
            utils.clear_display()
            
            # add nan subjects to list if not None
            if sub_id_if_nan is not None:
                nan_subjects.append(sub_id_if_nan)

Converting forward solution to surface orientation
    No patch info available. The standard source space normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Computing inverse operator with 59 channels.
    59 out of 62 channels remain after picking
Selected 59 channels
Creating the depth weighting matrix...
    59 EEG channels
    limit = 20485/20484 = 2.317813
    scale = 103014 exp = 0.8
Applying loose dipole orientations to surface source spaces: 0.2
Whitening the forward solution.
    Created an SSP operator (subspace dimension = 1)
Computing rank from covariance with rank=None
    Using tolerance 0.04 (2.2e-16 eps * 59 dim * 3.1e+12  max singular value)
    Estimated rank (eeg): 58
    EEG: rank 58 computed from 59 data channels with 1 projector
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of

In [ ]:
nan_subjects